# IBM Apliend data science capstone

## Segmenting and Clustering Neighborhoods in Toronto
### Part 2

We will convert addresses into their equivalent latitude and longitude values. Also, we will use the Foursquare API to explore neighborhoods in Toronto city.

We will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. 

We will use the _k_-means clustering algorithm to complete this task.

Finally, you will use the Folium library to visualize the neighborhoods in Toronto City and their emerging clusters.

In [1]:
from sklearn.cluster import KMeans

import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# Convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# Map rendering library
import folium 

import requests 
import json 

# Library to parse HTML and XML documents
from bs4 import BeautifulSoup 

# Transform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 